In [0]:
import pandas as pd
import numpy as np
import random
import pickle
import gc
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

In [30]:
#!git clone https://github.com/labs15-baseball-pitch-predictor/RC-v1.1--Data-Cleaning-and-Feature-Engineering.git

fatal: destination path 'RC-v1.1--Data-Cleaning-and-Feature-Engineering' already exists and is not an empty directory.


## Load in the pre-pitcher preprocessed_df



In [0]:
#load in pickled preprocessed df

df1 = pd.read_pickle("/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/pre_pitcher_pre_processed_df_part1.pkl", compression='zip')
df2 = pd.read_pickle("/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/pre_pitcher_pre_processed_df_part2.pkl", compression='zip')

df = pd.concat([df1, df2])

In [32]:
#clear some memory:
del df1
del df2
gc.collect()

868

In [0]:
#make list of pitchers (for filling in Nans in batting stat later for pitcher slot in batting order)

def make_pitcher_list(pre_processed_df):
    #make list of all the pitcher ids in the preprocessed_df
    pitcher_list = df['pitcher'].unique().tolist()
    
    #on rare occassions in extra-inning games, a position player is called in to pitch: 
    #to avoid categorizing him as a pitcher we'll only count pitchers with over 100 pitches:
    for pitcher in pitcher_list:
        if len(df[df['pitcher'] == pitcher]) < 100:
            pitcher_list.remove(pitcher)
    return pitcher_list

pitcher_list = make_pitcher_list(df)

## Choose the pitcher to model

In [0]:
#df.groupby('pitcher').count().sort_values('pitch_type', ascending=False).head(10)

In [137]:
#dictionary mapping top 10 pitchers with most pitches name with pitcher id

pitcher_id_dict = {'Justin Verlander': 434378.0, 'Trevor Bauer': 545333.0, 
                   'Aaron Nola': 605400.0, 'Lance Lynn': 458681.0, 'Gerrit Cole':
                   543037.0, 'Jacob deGrom':594798.0, 'Zack Greinke':425844.0,
                   'Patrick Corbin': 571578.0, 'Max Scherzer': 453286.0,
                   'Kyle Gibson': 502043.0}

#make pitcher_df subset where pitcher == picther_id
# pitcher_id = pitcher_id_dict['Justin Verlander']
# pitcher_id = pitcher_id_dict['Trevor Bauer']
# pitcher_id = pitcher_id_dict['Aaron Nola']
# pitcher_id = pitcher_id_dict['Lance Lynn']
# pitcher_id = pitcher_id_dict['Gerrit Cole']
pitcher_id = pitcher_id_dict['Jacob deGrom']
# pitcher_id = pitcher_id_dict['Zack Greinke']
# pitcher_id = pitcher_id_dict['Patrick Corbin']
# pitcher_id = pitcher_id_dict['Max Scherzer']
# pitcher_id = pitcher_id_dict['Kyle Gibson']

pitcher_df = df[df['pitcher'] == pitcher_id]
pitcher_df.head()

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat
11131,FT,2018-03-31,15419,94.900002,-1.2921,5.2625,451594.0,594798.0,2.0,NaN,2018.0,-1.0743,1.0163,0.2704,2.7180,0.0,0.0,0.0,0.0,1.0,NaN,NaN,518595.0,6.6515,-138.201904,-3.3637,-15.5889,31.759800,-18.2847,3.2540,1.4503,NaN,NaN,NaN,96.147003,2256.0,7.033,529443.0,53.466499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,R,NYM,STL,S,NaN,Top,180331_171036,Standard,0.0,NaN,False,False,False,False,0,0,1,0,64.454735,24.435591,39.745918,26.6,0.418005,0.196875,0.184375,21.141975,22.672064,35.099339,42.07317,0.393189,0.155556,0.188889,14.403293,35.294117,33.088234,24.731182,0.390467,0.188406,0.231884,0.0,0,0,Jacob deGrom,NaN,called_strike,NaN,2-Seam Fastball,Standard,00,neutral,fastball
11132,CH,2018-03-31,15397,88.099998,-1.1648,5.2164,451594.0,594798.0,13.0,NaN,2018.0,-1.0278,0.3822,-1.3268,1.7102,0.0,0.0,0.0,0.0,1.0,NaN,NaN,518595.0,1.8636,-128.468201,-3.2951,-12.1318,28.249201,-27.2638,3.5272,1.4602,NaN,NaN,NaN,89.766998,1587.0,7.340,529443.0,53.159599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,R,NYM,STL,B,NaN,Top,180331_171048,Standard,0.0,NaN,False,False,True,False,0,0,0,0,64.454735,24.435591,39.745918,26.6,0.418005,0.196875,0.184375,21.141975,22.672064,35.099339,42.07317,0.393189,0.155556,0.188889,14.403293,35.294117,33.088234,24.731182,0.390467,0.188406,0.231884,0.0,0,1,Jacob deGrom,NaN,ball,NaN,Changeup,Standard,01,ahead,offspeed
11133,FF,2018-03-31,15395,95.400002,-1.1504,5.3778,451594.0,594798.0,12.0,NaN,2018.0,-0.5913,1.3216,1.0642,3.0179,0.0,0.0,0.0,0.0,1.0,NaN,NaN,518595.0,7.2361,-138.735397,-3.5764,-9.4372,33.221001,-14.3048,3.3372,1.4390,235.0,82.400002,23.0,95.897003,2384.0,6.769,529443.0,53.730301,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,R,NYM,STL,S,NaN,Top,180331_171107,Standard,0.0,NaN,False,False,False,True,0,1,0,1,64.454735,24.435591,39.745918,26.6,0.418005,0.196875,0.184375,21.141975,22.672064,35.099339,42.07317,0.393189,0.155556,0.188889,14.403293,35.294117,33.088234,24.731182,0.390467,0.188406,0.231884,0.0,1,1,Jacob deGrom,NaN,foul,NaN,4-Seam Fastball,Standard,11,ahead,fastball
11134,SL,2018-03-31,15371,91.300003,-1.3156,5.4376,451594.0,594798.0,4.0,NaN,2018.0,0.4093,0.5042,-0.4831,2.6213,0.0,0.0,0.0,0.0,1.0,NaN,NaN,518595.0,1.2152,-133.035706,-2.2825,4.5804,30.456600,-25.8887,3.3372,1.4390,212.0,79.199997,21.0,91.508003,2502.0,6.493,529443.0,54.006901,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,R,NYM,STL,S,NaN,Top,180331_171129,Standard,0.0,NaN,False,False,False,False,0,1,1,0,64.454735,24.435591,39.745918,26.6,0.418005,0.196875,0.184375,21.141975,22.672064,35.099339,42.0731

## Add the pitcher prior tendencies (Pitcher Scouting Report)

In [139]:
%%time
#read in 2017 pre_procesed step 1 pickle and filter for pitcher_id
df17 = pd.read_pickle('/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/preprocessed_2017.pkl', compression = 'zip')
pitcher_df17 = df17[df17['pitcher'] == pitcher_id]

CPU times: user 2.02 s, sys: 135 ms, total: 2.15 s
Wall time: 2.16 s


In [0]:
def get_left_right_pitch_tendencies(pitcher_df):
    #split the df into left hand and right handed batters
    left = pitcher_df[pitcher_df['stand'] == 'L'].copy()
    right = pitcher_df[pitcher_df['stand'] == 'R'].copy()
    
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    overall_left = left['pitch_type'].value_counts(normalize=True).to_dict()
    overall_right = right['pitch_type'].value_counts(normalize=True).to_dict()
    
    #initialize empty dict for count categories tendencies
    by_count_left = {}
    by_count_right = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        left_subset = left[left['count_cat'] == cat]
        right_subset = right[right['count_cat'] == cat]
        
        by_count_left[cat] = left_subset['pitch_type'].value_counts(normalize=True).to_dict()
        by_count_right[cat] = right_subset['pitch_type'].value_counts(normalize=True).to_dict()
        
    return overall_left, overall_right, by_count_left, by_count_right

def make_tendency_features(pitcher_df, overall_left, overall_right, by_count_left, by_count_right):
    #helper functions to vectorize w/ df.apply():
    def get_overall_left_perc(x):
        return overall_left[x] * 100
    def get_overall_right_perc(x):
        return overall_right[x] * 100
    def get_by_count_left_perc(x):
        try:
            return by_count_left[x][pitch_type] * 100
        except KeyError:
            return 0
    def get_by_count_right_perc(x):
        try:
            return by_count_right[x][pitch_type] * 100
        except KeyError:
            return 0
    
    left = pitcher_df[pitcher_df['stand'] == 'L'].copy()
    right = pitcher_df[pitcher_df['stand'] == 'R'].copy()
    
    pitch_types_left = overall_left.keys()
    pitch_types_right = overall_right.keys()
    
    #Left
    for pitch_type in pitch_types_left:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
      
        left[overall_feature] = pitch_type
        left[overall_feature] = left[overall_feature].apply(get_overall_left_perc)
        left[count_cat_feature] = left['count_cat'].apply(get_by_count_left_perc)
    #Right
    for pitch_type in pitch_types_right:
        overall_feature = 'overall_' + pitch_type + '_perc'
        count_cat_feature = 'count_cat_' + pitch_type + '_perc'
      
        right[overall_feature] = pitch_type
        right[overall_feature] = right[overall_feature].apply(get_overall_right_perc)
        right[count_cat_feature] = right['count_cat'].apply(get_by_count_right_perc)
        
    
    return pd.concat([left,right]).sort_values(by=['game_date', 'game_pk', 'at_bat_number', 'pitch_number'])

def add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates):
    df = pd.concat([pitcher_df, pitcher_df17], sort=False)
    
    #initialize empty list to store dfs (concat them together later)
    df_list = []
    
    #iterate over each period
    for i in range(len(start_dates)):
      
        #make the prior and current dfs:
        prior_df = df[df['game_date'] < start_dates[i]]
        current_df = df[(df['game_date'] >= start_dates[i]) & (df['game_date'] <= end_dates[i])]
        
        #get the pitch tendencies from prior:
        overall_left, overall_right, by_count_left, by_count_right = get_left_right_pitch_tendencies(prior_df)
        
        #make the pitch tendencies features on current:
        current_df = make_tendency_features(current_df, overall_left, overall_right, by_count_left, by_count_right)
        
        #append the df to the list
        df_list.append(current_df)
    
    df = pd.concat(df_list, sort=False)
    return df


In [0]:

start_dates = ['2018-03-29', '2018-05-01', '2018-06-01', '2018-07-01', '2018-08-01', 
               '2018-09-01', '2019-03-28', '2019-05-01', '2019-06-01', '2019-07-01', 
               '2019-08-01']

end_dates =  ['2018-04-30', '2018-05-31', '2018-06-30', '2018-07-31', '2018-08-31', 
              '2018-10-01', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31', 
              '2019-08-31']

In [142]:
%%time
pitcher_df = add_pitcher_scouting_report(pitcher_df, pitcher_df17, start_dates, end_dates)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



CPU times: user 1.58 s, sys: 10 ms, total: 1.59 s
Wall time: 1.59 s


## Add Game Features:

#### Batting Order, Game pitch_count, & Trailing Pitch Features

In [0]:
def make_game_batting_order(game_df):
    game_df = game_df.sort_values(by=['at_bat_number', 'pitch_number'])
    all_batters = game_df['batter'].unique().tolist()
    #re-set the at_bat_number for the game to be sequential starting at 1
    at_bat_keys = game_df['at_bat_number'].unique().tolist()
    at_bat_values = range(1, len(at_bat_keys)+1)
    at_bat_map = dict(zip(at_bat_keys, at_bat_values))
    game_df['at_bat_number'] = game_df['at_bat_number'].replace(at_bat_map)
    
    #get the first 9 batter ids
    first_9_batter_subset = game_df[game_df['at_bat_number'] < 10]
    first_9_batters = first_9_batter_subset['batter'].unique().tolist()
    
    #map the batter id to batting order position 1-9
    batting_order_map = dict(zip(first_9_batters, range(1,10)))
    
    #for anyone else who bats later in the game, assign 'PH' (pinch hitter) to their batting order slot
    other_batters = list(set(all_batters) - set(first_9_batters))
    if len(other_batters) > 0:
        for batter in other_batters:
            batting_order_map[batter] = 'PH'
    try:
        game_df['batting_order_slot'] = game_df['batter'].apply(lambda x: batting_order_map[x])    
    except KeyError:
        game_df = None
        return game_df
    
    game_df['pitcher_AB'] = game_df['batter'].apply(lambda x: True if x in pitcher_list else False)
    game_df['batting_order_slot'] = game_df['batting_order_slot'].where(game_df['pitcher_AB'] == False, other='pitcher')
    return game_df

def get_pitch_tendencies(pitcher_df):
    #assign the normalized value counts for this pitchers pitch types to a dictionary
    pitcher_tendencies_overall = pitcher_df['pitch_type'].value_counts(normalize=True).to_dict()

    #initialize empty dict for count categories tendencies
    pitcher_tendencies_by_count = {}
    
    #loop over each count category and get the pitchers tendencies and add to the dict
    for cat in pitcher_df['count_cat'].unique().tolist():
        subset = pitcher_df[pitcher_df['count_cat'] == cat]
        pitcher_tendencies_by_count[cat] = subset['pitch_type'].value_counts(normalize=True).to_dict()
    return pitcher_tendencies_overall, pitcher_tendencies_by_count

def make_game_pitchcount_and_trailing_pitch_features_and_batting_order(pitcher_df, pitcher_list):
    df = pitcher_df.copy()
    all_games = []
    
    print('#pitches in df before: ' + str(len(df)))
    
    pitcher_tendencies_overall, pitcher_tendencies_by_count = get_pitch_tendencies(df)
    games = df['game_pk'].unique().tolist()
    
    for game in games:
        
        #take the first game and make the pitch count feature
        game_df = df[df['game_pk'] == game].copy()
        game_df['pitch_count'] = range(1, game_df.shape[0] + 1)
    
        #make the L1_pitch type feature:
        game_df['L1_pitch_type'] = game_df['pitch_type'].shift(periods=1)
        game_df['L1_pitch_result'] = game_df['type'].shift(periods=1)
        game_df['L1_pitch_result'] = game_df['L1_pitch_result'].replace({np.nan:'first pitch'})
        game_df['L1_pitch_zone'] = game_df['zone'].shift(periods=1)
        game_df['L1_pitch_zone'] = game_df['L1_pitch_zone'].fillna(-1)

        #overall strike % (to fill in for first 5 pitches L5_strike_perc)
        overall_strike_perc = df['type'].value_counts(normalize=True)['S'] * 100

        #make the trailing 5 pitches:
        for index, row in game_df.iterrows():
            #fill NaNs for L1_pitch using same method as when pitch_type was missing
            if row['pitch_count'] == 1:
                random_pitch = random.choices(population=list(pitcher_tendencies_overall.keys()), 
                                   weights=list(pitcher_tendencies_overall.values()), 
                                   k=1)[0]
                game_df.at[index, 'L1_pitch_type'] = random_pitch

            #for the first 5 rows, use overall pitcher tendencies    
            if row['pitch_count'] < 6:
                #fill with overall tendencies
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100
                    feature = 'L15_' + pitch + '_perc'
                    game_df.at[index, feature] = pitcher_tendencies_overall[pitch] * 100
                #strike %
                game_df.at[index, 'L5_strike_perc'] = overall_strike_perc
                game_df.at[index, 'L15_strike_perc'] = overall_strike_perc

            else:
                current_pitch = game_df.at[index, 'pitch_count']
                #make a subset of the prev 5 pitches
                subset = game_df[(game_df['pitch_count'] > current_pitch - 6) & (game_df['pitch_count'] < current_pitch)]
                #grab the value count percentages for the last 5 pitches
                subset_percentages = subset['pitch_type'].value_counts(normalize=True).to_dict()
                try:
                    L5_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                except KeyError:
                    L5_strike_perc = 0

                game_df.at[index, 'L5_strike_perc'] = L5_strike_perc

                #iterate over all possible pitch types this pitcher throws:
                for pitch in list(pitcher_tendencies_overall.keys()):
                    feature = 'L5_' + pitch + '_perc'
                    #if he has thrown that pitch type in last 5
                    try:
                        game_df.at[index, feature] = subset_percentages[pitch] * 100
                    #except for when he hasnt thrown that type in last 5
                    except:
                        game_df.at[index, feature] = 0


                if row['pitch_count'] < 16:
                    #make a subset of the prev 15 pitches
                    subset = game_df[(game_df['pitch_count'] < current_pitch)]
                    #grab the value count percentages for the last 15 pitches
                    subset_percentages = subset['pitch_type'].value_counts(normalize=True).to_dict()
                    try:
                        L15_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                    except KeyError:
                        L15_strike_perc = 0

                    game_df.at[index, 'L15_strike_perc'] = L15_strike_perc

                    #iterate over all possible pitch types this pitcher throws:
                    for pitch in list(pitcher_tendencies_overall.keys()):
                        feature = 'L15_' + pitch + '_perc'
                        #if he has thrown that pitch type in last 15
                        try:
                            game_df.at[index, feature] = subset_percentages[pitch] * 100
                        #except for when he hasnt thrown that type in last 5
                        except:
                            game_df.at[index, feature] = 0
                else:
                    #make a subset of the prev 15 pitches
                    subset = game_df[(game_df['pitch_count'] > current_pitch - 16) & (game_df['pitch_count'] < current_pitch)]
                    #grab the value count percentages for the last 5 pitches
                    subset_percentages = subset['pitch_type'].value_counts(normalize=True).to_dict()
                    try:
                        L15_strike_perc = subset['type'].value_counts(normalize=True)['S'] * 100
                    except KeyError:
                        L15_strike_perc = 0

                    game_df.at[index, 'L15_strike_perc'] = L15_strike_perc

                    #iterate over all possible pitch types this pitcher throws:
                    for pitch in list(pitcher_tendencies_overall.keys()):
                        feature = 'L15_' + pitch + '_perc'
                        #if he has thrown that pitch type in last 5
                        try:
                            game_df.at[index, feature] = subset_percentages[pitch] * 100
                        #except for when he hasnt thrown that type in last 5
                        except:
                            game_df.at[index, feature] = 0
                            
        #apply the battting order features to the game:                
        game_df = make_game_batting_order(game_df)
    
        all_games.append(game_df)
    
    new_df = pd.concat(all_games).sort_values(by=['game_date', 'game_pk', 'at_bat_number', 'pitch_number'])
    
    print('# pitches in df after: ' + str(len(new_df)))
    
    return new_df

def make_prev_ab_walk_basehit_run_and_homerun_features(pitcher_df):
    
    all_games = []
    #iterate over each game
    for game in pitcher_df['game_pk'].unique():
        #make subset df for that game
        game_df = pitcher_df[pitcher_df['game_pk'] == game].copy()
        #initialize columns to False:
        game_df['prev_ab_run_scored'] = False
        game_df['prev_ab_homerun'] = False
        game_df['prev_ab_walk'] = False
        game_df['prev_ab_basehit'] = False
        game_df['prev_ab_strikeout'] = False
        
        #this gets the 
        at_bats = game_df['at_bat_number'].sort_values().unique()
        
        #initialize empty dicts
        run_scored = []
        homeruns = []
        walks = []
        basehits = []
        strikeouts = []
        
        walks = ['walk', 'hit_by_pitch']
        basehits = ['single', 'double', 'triple', 'home_run']
        
        #starting w/ 2nd AB, iterate thru to the end of the at_bats:
        for ab in at_bats[2:]:
            #get the index for the last pitch of the prev AB
            prev_ab_last_pitch_index = game_df[game_df['at_bat_number'] == ab-1]['pitch_number'].index.max()
            #check if the last pitch resulted in a walk or hit by pitch:
            if game_df.loc[prev_ab_last_pitch_index]['events'] in walks:
                #if so, add an entry
                walks.append(ab)
            #check if last pitch gave up a basehit:
            elif game_df.loc[prev_ab_last_pitch_index]['events'] in basehits:
                basehits.append(ab)
            elif game_df.loc[prev_ab_last_pitch_index]['events'] == 'strikeout':
                strikeouts.append(ab)
            
            #to check if prev AB resulted in a run scoring: compare score before and after the AB
            prev_score = game_df[game_df['at_bat_number'] == ab-1]['bat_score'].values[0]
            current_score = game_df[game_df['at_bat_number'] == ab]['bat_score'].values[0]
            
            if current_score > prev_score:
                run_scored.append(ab)
                 
                #check if last AB gave up a homerun:
                if game_df.loc[prev_ab_last_pitch_index]['events'] == 'home_run':
                    homeruns.append(ab)
                    
        #iterate over each at_bat, and add the features to the df where appropriate
        for ab in at_bats:
            idx = game_df[game_df['at_bat_number'] == ab].index
            if ab in walks:
                game_df.at[idx, 'prev_ab_walk'] = True
            elif ab in basehits:
                game_df.at[idx, 'prev_ab_basehit'] = True
            elif ab in strikeouts:
                game_df.at[idx, 'prev_ab_strikeout'] = True
            if ab in run_scored:
                game_df.at[idx, 'prev_ab_run_scored'] = True
                if ab in homeruns:
                    game_df.at[idx, 'prev_ab_homerun'] = True
        all_games.append(game_df)
        
    return pd.concat(all_games).sort_values(by=['game_date', 'game_pk', 'pitch_count'])

In [144]:
%%time
pitcher_df = make_game_pitchcount_and_trailing_pitch_features_and_batting_order(pitcher_df, pitcher_list)

#pitches in df before: 5904
# pitches in df after: 5904
CPU times: user 1min 17s, sys: 16.8 ms, total: 1min 17s
Wall time: 1min 17s


In [145]:
%%time
pitcher_df = make_prev_ab_walk_basehit_run_and_homerun_features(pitcher_df)

CPU times: user 21.4 s, sys: 13 ms, total: 21.4 s
Wall time: 21.4 s


In [0]:
#pitcher_df[['game_pk','at_bat_number','bat_score','events', 'prev_ab_run_scored', 'prev_ab_homerun', 'prev_ab_walk', 'prev_ab_basehit', 'prev_ab_strikeout']].head(10)

In [147]:
cols = ['pitch_count', 'pitch_type', 'type', 'L1_pitch_type', 'L1_pitch_result', 'L1_pitch_zone','batting_order_slot']	

for col in pitcher_df.columns:
    if col[-4:] == 'perc':
        if col[:2]=='L5':
            cols.append(col)
        if col[:3]=='L15':
            cols.append(col)
        
pitcher_df[cols].head(20)

,pitch_count,pitch_type,type,L1_pitch_type,L1_pitch_result,L1_pitch_zone,batting_order_slot,L5_FF_perc,L15_FF_perc,L5_SL_perc,L15_SL_perc,L5_CH_perc,L15_CH_perc,L5_CU_perc,L15_CU_perc,L5_FT_perc,L15_FT_perc,L5_strike_perc,L15_strike_perc
11131,1,FT,S,FF,first pitch,-1.0,1,45.426829,45.426829,27.422087,27.422087,15.836721,15.836721,5.792683,5.792683,5.52168,5.521680,52.913279,52.913279
11132,2,CH,B,FT,S,2.0,1,45.426829,45.426829,27.422087,27.422087,15.836721,15.836721,5.792683,5.792683,5.52168,5.521680,52.913279,52.913279
11133,3,FF,S,CH,B,13.0,1,45.426829,45.426829,27.422087,27.422087,15.836721,15.836721,5.792683,5.792683,5.52168,5.521680,52.913279,52.913279
11134,4,SL,S,FF,S,12.0,1,45.426829,45.426829,27.422087,27.422087,15.836721,15.836721,5.792683,5.792683,5.52168,5.521680,52.913279,52.913279
11135,5,FF,S,SL,S,4.0,1,45.426829,45.426829,27.422087,27.422087,15.836721,15.836721,5.792683,5.792683,5.52168,5.521680,52.913279,52.913279
11136,6,SL,B,FF,S,12.0,1,40.000000,40.000000,20.000000,20.000000,20.000000,20.000000,0.000000,0.000000,20.00000,20.000000,80.000000,80.000000
11137,7,CH,B,SL,B,9.0,1,40.000000,33.333333,40.000000,33.333333,20.000000,16.666667,0.000000,0.000000,0.00000,16.666667,60.000000,66.666667
11138,8,FF,S,CH,B,13.0,1,40.000000,28.571429,40.000000,28.571429,20.000000,28.571429,0.000000,0.000000,0.00000,14.285714,60.000000,57.142857
11139,9,SL,S,FF,S,3.0,1,40.000000,37.500000,40.000000,25.000000,20.000000,25.000000,0.000000,0.000000,0.00000,12.500000,60.000000,62.500000
11140,10,FF,S,SL,S,8.0,2,40.000000,33.333333,40.000000,33.333333,20.000000,22.222222,0.000000,0.000000,0.00000,11.111111,60.000000,66.666667


In [148]:
pitcher_df[cols].tail(20)

,pitch_count,pitch_type,type,L1_pitch_type,L1_pitch_result,L1_pitch_zone,batting_order_slot,L5_FF_perc,L15_FF_perc,L5_SL_perc,L15_SL_perc,L5_CH_perc,L15_CH_perc,L5_CU_perc,L15_CU_perc,L5_FT_perc,L15_FT_perc,L5_strike_perc,L15_strike_perc
1315064,81,CH,B,SL,X,14.0,pitcher,20.0,13.333333,80.0,66.666667,0.0,20.000000,0.0,0.0,0.0,0.0,60.0,53.333333
1315065,82,CH,S,CH,B,11.0,pitcher,20.0,13.333333,60.0,60.000000,20.0,26.666667,0.0,0.0,0.0,0.0,60.0,46.666667
1315066,83,FF,S,CH,S,4.0,pitcher,20.0,13.333333,40.0,60.000000,40.0,26.666667,0.0,0.0,0.0,0.0,60.0,53.333333
1315067,84,CH,S,FF,S,1.0,pitcher,40.0,13.333333,20.0,60.000000,40.0,26.666667,0.0,0.0,0.0,0.0,60.0,60.000000
1315068,85,FF,S,CH,S,11.0,pitcher,20.0,13.333333,20.0,53.333333,60.0,33.333333,0.0,0.0,0.0,0.0,60.0,66.666667
1315069,86,SL,X,FF,S,1.0,pitcher,40.0,20.000000,0.0,53.333333,60.0,26.666667,0.0,0.0,0.0,0.0,80.0,66.666667
1315070,87,SL,S,SL,X,3.0,6,40.0,20.000000,20.0,60.000000,40.0,20.000000,0.0,0.0,0.0,0.0,80.0,66.666667
1315071,88,FF,B,SL,S,12.0,6,40.0,20.000000,40.0,60.000000,20.0,20.000000,0.0,0.0,0.0,0.0,80.0,66.666667
1315072,89,SL,S,FF,B,12.0,6,40.0,26.666667,40.0,53.333333,20.0,20.000000,0.0,0.0,0.0,0.0,60.0,60.000000
1315073,90,SL,X,SL,S,14.0,6,40.0,26.666667,60.0,53.333333,0.0,20.000000,0.0,0.0,0.0,0.0,60.0,60.000000


## Fill Batting Scouting Report NaNs

#### Load in the batting_order_slot map pickle

In [0]:
batting_order_slot_map = pickle.load(open("/content/RC-v1.1--Data-Cleaning-and-Feature-Engineering/data/batting_order_slot_map.pkl", "rb"))

In [150]:
batting_order_slot_map

{1: {'breaking_babip': 0.1872640385570563,
  'breaking_bip_swung_perc': 36.89605241827341,
  'breaking_chase_perc': 31.577836069125617,
  'breaking_est_woba': 0.3397664965875092,
  'breaking_iso_value': 0.13101442385072887,
  'breaking_perc_faced': 26.340695895944986,
  'breaking_taken_strike_perc': 35.5666746896674,
  'fastball_babip': 0.2154025865714933,
  'fastball_bip_swung_perc': 40.45598656570413,
  'fastball_chase_perc': 27.84080495082605,
  'fastball_est_woba': 0.37223512074559534,
  'fastball_iso_value': 0.15691701622442916,
  'fastball_perc_faced': 62.765084187795196,
  'fastball_taken_strike_perc': 31.962136660953938,
  'offspeed_babip': 0.20294449910457169,
  'offspeed_bip_swung_perc': 40.37310697509579,
  'offspeed_chase_perc': 36.54483903027876,
  'offspeed_est_woba': 0.32781539532817017,
  'offspeed_iso_value': 0.13571896819088838,
  'offspeed_perc_faced': 10.883704814573973,
  'offspeed_taken_strike_perc': 23.32780224123913,
  'pitchout_perc_faced': 0.010515101697300902

In [0]:
batter_cols = ['fastball_perc_faced','fastball_chase_perc','fastball_bip_swung_perc', 'fastball_taken_strike_perc',
               'fastball_est_woba', 'fastball_babip', 'fastball_iso_value', 'breaking_perc_faced', 'breaking_chase_perc',
               'breaking_bip_swung_perc', 'breaking_taken_strike_perc', 'breaking_est_woba', 'breaking_babip', 
               'breaking_iso_value', 'offspeed_perc_faced', 'offspeed_chase_perc', 'offspeed_bip_swung_perc',
               'offspeed_taken_strike_perc', 'offspeed_est_woba', 'offspeed_babip', 'offspeed_iso_value',
               'pitchout_perc_faced']

def fill_batting_nans(pitcher_df, batting_order_slot_map):
    df = pitcher_df.copy()
    for slot in df['batting_order_slot'].unique().tolist():
        subset = df[df['batting_order_slot'] == slot].copy()
        df = df.drop(subset.index)
        for col in batter_cols:
            subset[col] = subset[col].fillna(batting_order_slot_map[slot][col])
        df = pd.concat([df, subset])
        print('finished w/ slot: ' + str(slot))
    df = df.sort_values(by=['game_date', 'game_pk', 'pitch_count'])
    return df

In [152]:
%%time
pitcher_df = fill_batting_nans(pitcher_df, batting_order_slot_map)

finished w/ slot: 1
finished w/ slot: 2
finished w/ slot: 3
finished w/ slot: 4
finished w/ slot: 5
finished w/ slot: 6
finished w/ slot: 7
finished w/ slot: 8
finished w/ slot: pitcher
finished w/ slot: 9
finished w/ slot: PH
CPU times: user 494 ms, sys: 3.99 ms, total: 498 ms
Wall time: 505 ms


In [153]:
pitcher_df.isna().sum()

pitch_type                            0
game_date                             0
index                                 0
release_speed                         3
release_pos_x                         3
release_pos_z                         3
batter                                0
pitcher                               0
zone                                  3
hit_location                       4536
game_year                             0
pfx_x                                 3
pfx_z                                 3
plate_x                               3
plate_z                               3
on_3b                                 0
on_2b                                 0
on_1b                                 0
outs_when_up                          0
inning                                0
hc_x                               4993
hc_y                               4993
fielder_2                             0
vx0                                   3
vy0                                   3


## Add Pitcher / Batter Matchup History Report

In [0]:
##Todo

## Choose features to use in model input vector, re-cast datatypes, encode categorical variables (onehot and binary), scale appropriately (Standard vs MinMax vs Robust), and check for any remaining NaNs and choose imputation strategy

#### Train/test Split (IMO use April/2017 - end of June/2019 as Train, and July/August 2019 as Test)

In [0]:
#Todo

In [156]:
pitcher_df.head(20)

,pitch_type,game_date,index,release_speed,release_pos_x,release_pos_z,batter,pitcher,zone,hit_location,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,stand,p_throws,home_team,away_team,type,bb_type,inning_topbot,sv_id,if_fielding_alignment,score_diff,on_1b_id,ball_high,ball_low,ball_left,ball_right,bases_loaded,batter_swung,in_strikezone,chased,fastball_perc_faced,fastball_chase_perc,fastball_bip_swung_perc,fastball_taken_strike_perc,fastball_est_woba,fastball_babip,fastball_iso_value,breaking_perc_faced,breaking_chase_perc,breaking_bip_swung_perc,breaking_taken_strike_perc,breaking_est_woba,breaking_babip,breaking_iso_value,offspeed_perc_faced,offspeed_chase_perc,offspeed_bip_swung_perc,offspeed_taken_strike_perc,offspeed_est_woba,offspeed_babip,offspeed_iso_value,pitchout_perc_faced,balls,strikes,player_name,events,description,des,pitch_name,of_fielding_alignment,_count,count_cat,pitch_cat,overall_FF_perc,count_cat_FF_perc,overall_SL_perc,count_cat_SL_perc,overall_FT_perc,count_cat_FT_perc,overall_CH_perc,count_cat_CH_perc,overall_CU_perc,count_cat_CU_perc,pitch_count,L1_pitch_type,L1_pitch_result,L1_pitch_zone,L5_FF_perc,L15_FF_perc,L5_SL_perc,L15_SL_perc,L5_CH_perc,L15_CH_perc,L5_CU_perc,L15_CU_perc,L5_FT_perc,L15_FT_perc,L5_strike_perc,L15_strike_perc,batting_order_slot,pitcher_AB,prev_ab_run_scored,prev_ab_homerun,prev_ab_walk,prev_ab_basehit,prev_ab_strikeout
11131,FT,2018-03-31,15419,94.900002,-1.2921,5.2625,451594.0,594798.0,2.0,NaN,2018.0,-1.0743,1.0163,0.2704,2.7180,0.0,0.0,0.0,0.0,1.0,NaN,NaN,518595.0,6.6515,-138.201904,-3.3637,-15.5889,31.759800,-18.284700,3.2540,1.4503,NaN,NaN,NaN,96.147003,2256.0,7.033,529443.0,53.466499,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,R,NYM,STL,S,NaN,Top,180331_171036,Standard,0.0,NaN,False,False,False,False,0,0,1,0,64.454735,24.435591,39.745918,26.600000,0.418005,0.196875,0.184375,21.141975,22.672064,35.099339,42.073170,0.393189,0.155556,0.188889,14.403293,35.294117,33.088234,24.731182,0.390467,0.188406,0.231884,0.000000,0,0,Jacob deGrom,NaN,called_strike,NaN,2-Seam Fastball,Standard,00,neutral,fastball,35.851472,31.944444,23.815621,22.916667,14.852753,19.965278,14.148528,9.895833,11.331626,15.277778,1,FF,first pitch,-1.0,45.426829,45.426829,27.422087,27.422087,15.836721,15.836721,5.792683,5.792683,5.52168,5.521680,52.913279,52.913279,1,False,False,False,False,False,False
11132,CH,2018-03-31,15397,88.099998,-1.1648,5.2164,451594.0,594798.0,13.0,NaN,2018.0,-1.0278,0.3822,-1.3268,1.7102,0.0,0.0,0.0,0.0,1.0,NaN,NaN,518595.0,1.8636,-128.468201,-3.2951,-12.1318,28.249201,-27.263800,3.5272,1.4602,NaN,NaN,NaN,89.766998,1587.0,7.340,529443.0,53.159599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L,R,NYM,STL,B,NaN,Top,180331_171048,Standard,0.0,NaN,False,False,True,False,0,0,0,0,64.454735,24.435591,39.745918,26.600000,0.418005,0.196875,0.184375,21.141975,22.672064,35.099339,42.073170,0.393189,0.155556,0.188889,14.403293,35.294117,33.088234,24.731182,0.390467,0.188406,0.231884,0.000000,0,1,Jacob deGrom,NaN,ball,NaN,Changeup,Standard,01,ahead,offspeed,35.851472,40.129870,23.815621,24.675325,14.852753,9.350649,14.148528,15.844156,11.331626,10.000000,2,FT,S,2.0,45.426829,45.426829,27.422087,27.422087,15.836721,15.836721,5.792683,5.792683,5.52168,5.521680,52.913279,52.913279,1,False,False,False,False,False,False
11133,FF,2018-03-31,15395,95.400002,-1.1504,5.3778,451594.0,594798.0,12.0,NaN,2018.0,-0.5913,1.3216,1.0642,3.0179,0.0,0.0,0.0,0.0,1.0,NaN,NaN,518595.0,7.2361,-138.735397,-3.5764,-9.4372,33.22